<a href="https://colab.research.google.com/github/MGonzalez506/Dimensionamiento_PV/blob/10-limpiar-un-poco-el-c%C3%B3digo-y-mejorar-rendimiento/Simulaci%C3%B3n_y_Dimensionamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Taller PVLIB

In [2]:
!pip install pvlib
!pip install rdtools

     |████████████████████████████████| 27.6 MB 77.7 MB/s 
     |████████████████████████████████| 947 kB 5.0 MB/s 
     |████████████████████████████████| 192.9 MB 63 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [3]:
#Imports
import sys
import os
import traceback
import time
import signal
import fcntl
import string
import re
import itertools

import pytz
import pvlib
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

from time import sleep
from datetime import datetime
from io import StringIO
from pvlib import solarposition
from pvlib import pvsystem
from pvlib import location
from pvlib import clearsky, atmosphere, solarposition
from pvlib.location import Location
from pvlib import irradiance
from pvlib import temperature
from rdtools import get_clearsky_tamb

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0]=12
fig_size[1]=8
plt.rcParams["figure.figsize"] = fig_size

site_tz='America/Costa_Rica'
site_lat = 9.84950
site_lon = -83.91289
site_name = 'Moren Centro de acondicionamiento Físico'
site_alt = 1425

glosario = {
      'sapm':'Sandia PV Array Performance Model',
      'sam':'System Advisor Model',
      'tmy':'Typical Meteorological Year Data',
      'ghi':'Radiación global Horizontal', 
      'dni':'Radiación directa', 
      'dhi':'Radiación difusa', 
      'IR(h)':'Índice de claridad', 
      'aoi':'Ángulo de incidencia', 
      'dni_extra':'Porcentaje de radiación extra',
      'POA':'Plane of Array'
      }

#Get modules & inverters
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
sapm_inverters = pvlib.pvsystem.retrieve_sam('CECInverter')

module = (sandia_modules['Canadian_Solar_CS5P_220M___2009_'])
inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

tmys = []
selected_months = []

coordinates = [
               (site_lat,site_lon,site_name,site_alt,site_tz)
]

In [5]:
def get_pvgis_tmy_from_coordinates(coordinates):
  tmys = []
  months = []
  for location in coordinates:
    latitude, longitude, name, altitude, timezone = location
    data_frame = pvlib.iotools.get_pvgis_tmy(latitude,longitude,map_variables=True)
    weather = data_frame[0]
    weather.index.name = 'utc_time'
    tmys.append(weather)
    months = data_frame[1]
  return tmys, months
tmys,selected_months = get_pvgis_tmy_from_coordinates(coordinates)

In [6]:
#Para realizar el cálculo del ángulo de inclinación óptima
system = {'module':module, 'inversor':inverter, 'surface_azimuth':180}
azimuth_testing = [0,90,180,270]
tilt_testing = [5,10,15]
energies = {}
monthly_energy = {}
last_ac = pd.DataFrame()
for tilt_angle in tilt_testing:
  for azimuth_angle in azimuth_testing:
    for weather in tmys:
        location = coordinates[0]
        latitude, longitude, name, altitude, timezone = location
        system['surface_tilt']=tilt_angle
        system['surface_azimuth']=azimuth_angle
        solpos = pvlib.solarposition.get_solarposition(
            time=weather.index,
            latitude=latitude,
            longitude=longitude,
            altitude=altitude,
            temperature=weather['temp_air'],
            pressure=pvlib.atmosphere.alt2pres(altitude)
        )
        dni_extra = pvlib.irradiance.get_extra_radiation(weather.index)
        airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
        pressure = pvlib.atmosphere.alt2pres(altitude)
        am_abs = pvlib.atmosphere.get_absolute_airmass(airmass,pressure)
        aoi = pvlib.irradiance.aoi(
            system['surface_tilt'],
            system['surface_azimuth'],
            solpos['apparent_zenith'],
            solpos['azimuth']
        )
        #Irradiación total general
        total_irradiance = pvlib.irradiance.get_total_irradiance(
          system['surface_tilt'],
          system['surface_azimuth'],
          solpos['apparent_zenith'],
          solpos['azimuth'],
          weather['dni'],
          weather['ghi'],
          weather['dhi'],
          dni_extra=dni_extra,
          model='haydavies'
        )
        cell_temperature = pvlib.temperature.sapm_cell(
          total_irradiance['poa_global'],
          weather['temp_air'],
          weather['wind_speed'],
          **temperature_model_parameters     
        )
        #Irradiación efectiva
        effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
          total_irradiance['poa_direct'],
          total_irradiance['poa_diffuse'],
          am_abs,
          aoi,
          module
        )  
        dc=pvlib.pvsystem.sapm(effective_irradiance,cell_temperature,module)
        ac=pvlib.inverter.sandia(dc['v_mp'],dc['p_mp'],inverter)
        last_ac = ac
        annual_energy = ac.sum()
        label = str(tilt_angle) + "º & " + str(azimuth_angle) + "Azi"
        monthly_energy[label] = ac
        energies[label] = annual_energy

In [41]:
#print(last_ac.keys())
print("###################")
date_min = datetime.strptime("2005-01-01 00:00:00+00:00","%Y-%m-%d %H:%M:%S%z")
date_max = datetime.strptime("2005-01-30 00:00:00+00:00","%Y-%m-%d %H:%M:%S%z")
#x = last_ac.where(last_ac.keys() < "2005-01-30 00:00:00+00:00")
#x = last_ac.between_time('00:00', '01:00') #Works but not to much
#x = last_ac.loc['2005-01-01':'2005-01-02'] #Working perfectly <--------------------
estudio_por_mes = []
for i in selected_months:
  this_year = i['year']
  this_month = i['month']
  mes = str(this_year) + '-' + str(this_month) + '-01'
  x = last_ac.loc[mes:mes]
  print(x)

###################
utc_time
2005-01-01 00:00:00+00:00     -0.075000
2005-01-01 01:00:00+00:00     -0.075000
2005-01-01 02:00:00+00:00     -0.075000
2005-01-01 03:00:00+00:00     -0.075000
2005-01-01 04:00:00+00:00     -0.075000
2005-01-01 05:00:00+00:00     -0.075000
2005-01-01 06:00:00+00:00     -0.075000
2005-01-01 07:00:00+00:00     -0.075000
2005-01-01 08:00:00+00:00     -0.075000
2005-01-01 09:00:00+00:00     -0.075000
2005-01-01 10:00:00+00:00     -0.075000
2005-01-01 11:00:00+00:00     -0.075000
2005-01-01 12:00:00+00:00     -0.075000
2005-01-01 13:00:00+00:00      5.924768
2005-01-01 14:00:00+00:00     39.636615
2005-01-01 15:00:00+00:00     89.311603
2005-01-01 16:00:00+00:00    128.515192
2005-01-01 17:00:00+00:00    154.215160
2005-01-01 18:00:00+00:00    167.523986
2005-01-01 19:00:00+00:00    167.914382
2005-01-01 20:00:00+00:00    154.835428
2005-01-01 21:00:00+00:00    127.568155
2005-01-01 22:00:00+00:00     84.012294
2005-01-01 23:00:00+00:00     22.066909
dtype: floa